![title](DEP_LOGO_2.jpg)

# Trend Station Water Quality Data 
## Web Application Demo

In [1]:
#HIDDEN

import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import csv
from geopy.geocoders import Nominatim
from time import sleep
import folium
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
import seaborn as sns
from scipy import stats

In [2]:
#HIDDEN

dfhead = pd.read_csv('gm_export_TS_Location.csv')
dfhead = dfhead.fillna(0)
dfheadloc = dfhead[dfhead.latitude != 0]

geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Flatwoods, WV")

longitude = location.longitude
latitude = location.latitude

# create map of West Virginia using latitude and longitude values
ts_locations = folium.Map(location=[latitude, longitude], tiles="OpenStreetMap", zoom_start=7.4)

# add markers to map
for lat, lng, label in zip(dfheadloc['latitude'], dfheadloc['longitude'], dfheadloc['station_id']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=True).add_to(ts_locations)  

ts_locations

## Trend Station Location

In [3]:
#HIDDEN

dfhead = pd.read_csv('gm_export_TS_Location.csv')
dfhead = dfhead.fillna(0)
dfheadloc = dfhead[dfhead.latitude != 0]


def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

dropdown_stations_id = widgets.Dropdown(options = unique_sorted_values(dfheadloc.station_id), description='Trend Station')

output_two = widgets.Output()



def common_filtering(station):
    
    output_two.clear_output()
    
    
    global common_filter
    common_filter = dfheadloc[(dfheadloc.station_id == station)]
    
    with output_two:
        display(common_filter)
        
    
        
def dropdown_stations_id_eventhandler(change):
    common_filtering(change.new)

    
dropdown_stations_id.observe(dropdown_stations_id_eventhandler, names='value')
    
display(dropdown_stations_id)


Dropdown(description='Trend Station', options=('TS001', 'TS002', 'TS003', 'TS004', 'TS005', 'TS006', 'TS007', …

In [4]:
#HIDDEN
display(output_two)

Output()

## Trend Station Data

In [5]:
#HIDDEN

df_comb = pd.read_csv('TS_New_Combined_Final.csv')
df_comb = df_comb.fillna(0)
#df_comb['test_batch_id'] = pd.to_datetime(df_comb['test_batch_id'], format="%m/%d/%Y")
df_comb['result_value'] = pd.to_numeric(df_comb['result_value'])
df_comb = df_comb.sort_values(by=['station_id', 'test_batch_id'])
df_comb = df_comb.set_index('test_batch_id')

ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

dropdown_stations = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_comb.station_id), description='Trend Station')
dropdown_param = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_comb.param_name), description='Parameter')

output_three = widgets.Output()
plot_output = widgets.Output()


def common_filtering1(station, param):
    
    output_three.clear_output()
    plot_output.clear_output()
    
    if (station == ALL) & (param == ALL):
        global common_filter1
        common_filter1 = df_comb
    elif (station == ALL):
        common_filter1 = df_comb[df_comb.param_name == param]
    elif (param == ALL):
        common_filter1 = df_comb[df_comb.station_id == station]
    else:
        common_filter1 = df_comb[(df_comb.station_id == station) & (df_comb.param_name == param)]
    
    with output_three:
        display(common_filter1)
        
    with plot_output:
        if param == ALL or station == ALL:
            print("No Available Plots for the Selected Station/Parameter Combination!")
        else:
            plt.figure(figsize=(20,8));
            plt.plot(common_filter1['result_value'], 'go--', linewidth=1, markersize=6);
            plt.grid();
            plt.ylabel(param, fontsize=15);
            plt.xlabel('Date', fontsize=15);
            plt.title(station, fontsize=25);
            plt.show();


def dropdown_stations_eventhandler(change):
    common_filtering1(change.new, dropdown_param.value)
    
def dropdown_param_eventhandler(change):
    common_filtering1(dropdown_stations.value, change.new)
    
dropdown_stations.observe(dropdown_stations_eventhandler, names='value')
dropdown_param.observe(dropdown_param_eventhandler, names='value')
    
display(dropdown_stations)
display(dropdown_param)

Dropdown(description='Trend Station', options=('ALL', 'TS001', 'TS002', 'TS003', 'TS004', 'TS005', 'TS006', 'T…

Dropdown(description='Parameter', options=('ALL', 'ALKALINITY', 'ALUMINUM', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM'…

In [9]:
#HIDDEN
display(output_three)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '              station_id    sample_id …

In [10]:
#HIDDEN
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
display(plot_output)

Output(outputs=({'output_type': 'stream', 'text': 'No Available Plots for the Selected Station/Parameter Combi…

## Trend Station Statistics

Need to convert DataFrame structure prior to this step!

In [31]:
common_filter1.reset_index(inplace=True)

group = common_filter1.groupby('param_name')
df2 = pd.DataFrame(group.apply(lambda x: x['test_batch_id'].unique()))

df2.reset_index(inplace=True)
df2 = df2.rename({0: 'DATES'}, axis='columns')
df2

,param_name,DATES
0,ALKALINITY,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
1,ALUMINUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
2,ANTIMONY,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
3,ARSENIC,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
4,BERYLLIUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
5,BI-CARBONATE,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
6,CADMIUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
7,CALCIUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
8,CARBONATE,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
9,CHROMIUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."


In [46]:
for row in df2.itertuples(index=False):
    print(row)

Pandas(param_name='ALKALINITY', DATES=array(['1/1/2016', '1/1/2017', '1/1/2018', '10/1/2016', '10/1/2017',
       '11/1/2016', '11/1/2017', '12/1/2015', '12/1/2016', '12/1/2017',
       '2/1/2016', '2/1/2017', '2/1/2018', '3/1/2016', '3/1/2017',
       '3/1/2018', '4/1/2016', '4/1/2017', '4/1/2018', '5/1/2016',
       '5/1/2017', '5/1/2018', '6/1/2016', '6/1/2017', '6/1/2018',
       '7/1/2016', '8/1/2016', '9/1/2016'], dtype=object))
Pandas(param_name='ALUMINUM', DATES=array(['1/1/2016', '1/1/2017', '1/1/2018', '10/1/2016', '10/1/2017',
       '11/1/2016', '11/1/2017', '12/1/2015', '12/1/2016', '12/1/2017',
       '2/1/2016', '2/1/2017', '2/1/2018', '3/1/2016', '3/1/2017',
       '3/1/2018', '4/1/2016', '4/1/2017', '4/1/2018', '5/1/2016',
       '5/1/2017', '5/1/2018', '6/1/2016', '6/1/2017', '6/1/2018',
       '7/1/2016', '8/1/2016', '9/1/2016'], dtype=object))
Pandas(param_name='ANTIMONY', DATES=array(['1/1/2016', '1/1/2017', '1/1/2018', '10/1/2016', '10/1/2017',
       '4/1/2016', 

In [48]:
for key,value in df2.iteritems():
    print(key,value)

param_name 0             ALKALINITY
1               ALUMINUM
2               ANTIMONY
3                ARSENIC
4              BERYLLIUM
5           BI-CARBONATE
6                CADMIUM
7                CALCIUM
8              CARBONATE
9               CHROMIUM
10                COPPER
11               CYANIDE
12      DISSOLVED SOLIDS
13    FIELD CONDUCTIVITY
14              FIELD ph
15                  FLOW
16           HOT ACIDITY
17                  IRON
18      LAB CONDUCTIVITY
19                LAB pH
20                  LEAD
21             MAGNESIUM
22             MANGANESE
23               MERCURY
24       MINERAL ACIDITY
25                NICKEL
26               NITRATE
27               PHENOLS
28             PHOSPHATE
29             POTASSIUM
30              SELENIUM
31                SILVER
32                SODIUM
33               SULFATE
34      SUSPENDED SOLIDS
35           TEMPERATURE
36              THALLIUM
37                  ZINC
Name: param_name, dtype: object
DATES 0

In [50]:
for row_index,row in df2.iterrows():
   print(row_index,row)

0 param_name                                           ALKALINITY
DATES         [1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1...
Name: 0, dtype: object
1 param_name                                             ALUMINUM
DATES         [1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1...
Name: 1, dtype: object
2 param_name                                             ANTIMONY
DATES         [1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1...
Name: 2, dtype: object
3 param_name                                              ARSENIC
DATES         [1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1...
Name: 3, dtype: object
4 param_name                                            BERYLLIUM
DATES         [1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1...
Name: 4, dtype: object
5 param_name                                         BI-CARBONATE
DATES         [1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1...
Name: 5, dtype: object
6 param_name                                              CADMIUM
DATES         [1